In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

In [5]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [6]:
start_year = 1985
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)
                        

2024-10-06 17:08:45
2024-10-06 17:18:27
2024-10-06 17:32:23
2024-10-06 17:48:05
2024-10-06 18:02:04
2024-10-06 18:15:16
2024-10-06 18:28:36
2024-10-06 18:40:51
2024-10-06 18:54:17
2024-10-06 19:07:36
2024-10-06 19:20:41
2024-10-06 19:34:16
2024-10-06 19:47:21
2024-10-06 20:01:57
2024-10-06 20:15:49
2024-10-06 20:29:05
2024-10-06 20:42:25
2024-10-06 20:54:06
2024-10-06 21:07:26
2024-10-06 21:19:23
2024-10-06 21:32:33
2024-10-06 21:44:55
2024-10-06 21:57:24
2024-10-06 22:10:07


In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1985
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:39:04
2024-11-02 20:50:15
2024-11-02 21:04:27
2024-11-02 21:17:41
2024-11-02 21:31:50
2024-11-02 21:44:52
2024-11-02 21:58:34
2024-11-02 22:11:30
2024-11-02 22:25:42
2024-11-02 22:39:09
2024-11-02 22:53:46
2024-11-02 23:07:07
2024-11-02 23:20:00
2024-11-02 23:33:44
2024-11-02 23:46:50
2024-11-03 00:00:48
2024-11-03 00:13:55
2024-11-03 00:26:27
2024-11-03 00:40:14
2024-11-03 00:53:41
2024-11-03 01:07:44
2024-11-03 01:21:52
2024-11-03 01:35:09
2024-11-03 01:49:23


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1985
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:02:44
2024-11-03 01:16:17
2024-11-03 01:30:25
2024-11-03 01:45:07
2024-11-03 02:00:03
2024-11-03 02:13:28
2024-11-03 02:26:47
2024-11-03 02:39:40
2024-11-03 02:53:08
2024-11-03 03:05:37
2024-11-03 03:19:12
2024-11-03 03:32:53
2024-11-03 03:46:03
2024-11-03 03:58:47
2024-11-03 04:12:14
2024-11-03 04:25:27
2024-11-03 04:39:14
2024-11-03 04:52:03


OSError: [Errno -101] NetCDF: HDF error: '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/765041.TD2.wrf2d_d01_1987-03-21_18:00:00.nc'

In [10]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1987
end_year = start_year + 1

for year in range(start_year, end_year):  # 1988 不包含
    months = range(3, 10)  # 3月到9月

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        # 设置每个月的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 检查闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29
            else:
                num_days = 28

        # 仅在1987年3月从21日开始
        start_day = 21 if (year == 1987 and month == 3) else 1
        end_day = 30 if (year == 1987 and month == 9) else num_days

        for day in range(start_day, end_day + 1):
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
                    adjusted_times = original_times + np.timedelta64(offset, 'h')
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset) + str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)


2024-11-03 12:35:42
2024-11-03 12:38:03
2024-11-03 12:44:48
2024-11-03 12:51:31
2024-11-03 12:57:43
2024-11-03 13:04:47
2024-11-03 13:11:46


In [2]:
import glob
def create_temp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_tmin = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_tmean = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_tmin,ds_tmean



def create_dtemp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_dtmin = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_dtmean = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_dtmin,ds_dtmean

base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern = 'T2.wrf2d_d01_????-??-??.nc'

folder_names = [

    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]


for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path = os.path.join(base_path, folder, file_pattern)
    all_files = glob.glob(full_path)
    #####
    summer_files = [f for f in all_files if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_summer = xr.open_mfdataset(summer_files)
    ds_jja = ds_summer.sel(time=ds_summer['time'].dt.month.isin([6, 7, 8]))
    lonn = ds_jja.lon.values
    latt = ds_jja.lat.values
    arr_t = ds_jja.t2.values

    ds_tmin,ds_tmean = create_temp_dataset(arr_t, latt, lonn)

    output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
    ds_tmin.to_netcdf(output_folder+'temp_min_'+folder+'.nc')
    ds_tmean.to_netcdf(output_folder+'temp_mean_'+folder+'.nc')

2024-11-03 21:13:52
2024-11-03 21:18:25
2024-11-03 21:23:23
2024-11-03 21:28:16
2024-11-03 21:33:26
2024-11-03 21:38:13
2024-11-03 21:43:13
2024-11-03 21:48:15
2024-11-03 21:53:34


In [2]:
import numpy as np
from datetime import datetime

def calculate_dn_averages(data, block_size=12):
    """
    计算每个时间块的平均温度值。

    参数:
    data -- 输入的温度数据数组。
    block_size -- 每个时间块的大小（默认为12个月）。

    返回:
    包含平均温度值的新数组，形状与输入数组相同。
    """
    # 创建一个形状相同的数组来存放结果，所有元素初始化为 NaN
    arr_t_dn = np.full_like(data, np.nan)
    
    # 循环处理每个位置的数据
    for i in range(data.shape[1]):

        for j in range(data.shape[2]):
            for k in range(data.shape[0] // block_size):
                # 选择第k个时间块的数据块进行平均
                arr_t_avg = np.mean(data[k*block_size:(k+1)*block_size, i, j])
                # 将平均结果填充到新数组的相应位置
                arr_t_dn[k*block_size:(k+1)*block_size, i, j] = arr_t_avg

    return arr_t_dn

# 使用示例：
# 假设 ds_selected 是你的数据集，并且已经加载了需要的数据。
# arr_t = ds_selected.t2.values
# 计算平均值
# result = calculate_monthly_averages(arr_t)

def cal_dn_t(array, dataset):
    lonn = dataset.lon.values
    latt = dataset.lat.values
    timee = dataset.time.values
    
    ds_dn_t = xr.Dataset(
        {'dnt': (['time', 'lat', 'lon'], array)},
        coords={
            'lon': (['lon'], lonn),
            'lat': (['lat'], latt),
            'time': ('time', timee)
        }
    )
    return ds_dn_t

In [11]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'
file_pattern_t = 'T2.wrf2d_d01_????-??-??.nc'
output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'

folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 
    'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]

In [12]:
import glob
for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path_t = os.path.join(base_path, folder, file_pattern_t)
    all_files_t = glob.glob(full_path_t)
    #####
    summer_files_t = [f for f in all_files_t if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f or '-05-' in f]
    ds = xr.open_mfdataset(summer_files_t)
    

    # 构建时间选择器，每年的5月31日18:00到9月1日05:00
    time_selection = (ds.time.dt.month == 5) & (ds.time.dt.day == 31) & (ds.time.dt.hour >= 18) | \
                     (ds.time.dt.month == 6) | \
                     (ds.time.dt.month == 7) | \
                     (ds.time.dt.month == 8) | \
                     ((ds.time.dt.month == 9) & (ds.time.dt.day == 1) & (ds.time.dt.hour <= 5))

    # 应用时间选择器
    ds_selected = ds.where(time_selection, drop=True)
    arr_t = ds_selected.t2.values
    arr_dnt = calculate_dn_averages(arr_t)
    ds_dnt = cal_dn_t(arr_dnt, ds_selected)
    ds_dnt_jja = ds_dnt.sel(time=ds_dnt['time'].dt.month.isin([6, 7, 8]))
    ds_dnt_jja.to_netcdf(output_folder+'dn_temp_'+folder+'.nc')

2024-12-27 16:15:11
2024-12-27 16:53:18
2024-12-27 17:30:22
2024-12-27 18:06:40
2024-12-27 18:43:43
2024-12-27 19:20:56
2024-12-27 19:58:04
2024-12-27 20:35:37


In [3]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'
file_pattern_t = 'TD2.wrf2d_d01_????-??-??.nc'
output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'

folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]

In [4]:
import glob
for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path_t = os.path.join(base_path, folder, file_pattern_t)
    all_files_t = glob.glob(full_path_t)
    #####
    summer_files_t = [f for f in all_files_t if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f or '-05-' in f]
    ds = xr.open_mfdataset(summer_files_t)
    

    # 构建时间选择器，每年的5月31日18:00到9月1日05:00
    time_selection = (ds.time.dt.month == 5) & (ds.time.dt.day == 31) & (ds.time.dt.hour >= 18) | \
                     (ds.time.dt.month == 6) | \
                     (ds.time.dt.month == 7) | \
                     (ds.time.dt.month == 8) | \
                     ((ds.time.dt.month == 9) & (ds.time.dt.day == 1) & (ds.time.dt.hour <= 5))

    # 应用时间选择器
    ds_selected = ds.where(time_selection, drop=True)
    arr_t = ds_selected.td2.values
    arr_dnt = calculate_dn_averages(arr_t)
    ds_dnt = cal_dn_t(arr_dnt, ds_selected)
    ds_dnt_jja = ds_dnt.sel(time=ds_dnt['time'].dt.month.isin([6, 7, 8]))
    ds_dnt_jja.to_netcdf(output_folder+'dn_dewtemp_'+folder+'.nc')

2024-12-29 00:50:19
2024-12-29 01:26:13
2024-12-29 02:02:27
2024-12-29 02:37:10
2024-12-29 03:12:07
2024-12-29 03:46:34
2024-12-29 04:21:48
2024-12-29 04:56:29
2024-12-29 05:31:45
